---
title: 日時モジュールの使い方
subject: python
keywords: [python, pendulum, time, datetime]
authors: [Shota Takahashi]
exports:
  - format: pdf
---

+++ {"part": "abstract"}
Python標準モジュールの``time``と``datetime``、
そして外部モジュールの``pendulum``を使って、
よくある日時の取り扱いを比べてみました。
+++

``time``と``datetime``は標準モジュールなので、そのまま読み込めます。
``pendulum``は``pip``などを使ってあらかじめ追加しておきます。
今回は最新版のv3系を使いました。

In [1]:
import time
import datetime
import pendulum
pendulum.__version__

'3.0.0'

# 現在時刻を取得したい

スクリプトを実行した時刻など、現在時刻を取得したい場合はよくあります。

1. ``datetime.datetime.now``
2. ``pendulum.now``
3. ``pendulum.from_timestamp``

In [2]:
dt = datetime.datetime.now()
print(dt)
dt

2024-01-26 19:55:34.989851


datetime.datetime(2024, 1, 26, 19, 55, 34, 989851)

``datetime.datetime.now``の引数に``datetime.timezone``を指定すると、タイムゾーン情報を持たせることができます。

In [3]:
dt = datetime.datetime.now(datetime.timezone(datetime.timedelta(hours=9)))
print(dt)
dt

2024-01-26 19:55:34.993708+09:00


datetime.datetime(2024, 1, 26, 19, 55, 34, 993708, tzinfo=datetime.timezone(datetime.timedelta(seconds=32400)))

``pendulum``はデフォルトでタイムゾーン情報を持った日時オブジェクトを作成します。

In [4]:
dt = pendulum.now()
print(dt)
dt

2024-01-26 19:55:34.997772+09:00


DateTime(2024, 1, 26, 19, 55, 34, 997772, tzinfo=Timezone('Asia/Tokyo'))

``time.time()``で現在のUNIX時間を取得できます。
``pendulum.from_timestamp``を使って``DateTime``オブジェクトに変換できます。

In [5]:
ut = time.time()
pendulum.from_timestamp(ut)

DateTime(2024, 1, 26, 10, 55, 35, 1553, tzinfo=Timezone('UTC'))

``datetime.datetime.now``で取得した日時オブジェクトは、``pendulum.instance()``で``DateTime``オブジェクトに変換できます。
もとの日時オブジェクトがタイムゾーン情報を持たない場合は``UTC``に変換されます。
タイムゾーン情報を持つ場合は、時間差情報が追加されます。

In [6]:
dt = datetime.datetime.now(datetime.timezone(datetime.timedelta(hours=9)))
pendulum.instance(dt)

DateTime(2024, 1, 26, 19, 55, 35, 5225, tzinfo=FixedTimezone(32400, name="+09:00"))

# 任意の日付したい

1. ``datetime.datetime``
2. ``pendulum.datetime``
3. ``pendulum.local``

``(年, 月, 日, 時, 分, 秒, マイクロ秒)``のタプルを与えて、任意の日時オクジェクトを作成できます。
``datetime``モジュールはタイムゾーン情報を持たないため、``tz-naive``なオブジェクトが作成されます。

In [7]:
dt = datetime.datetime(2024, 1, 2, 3, 45, 56)
print(dt)
dt

2024-01-02 03:45:56


datetime.datetime(2024, 1, 2, 3, 45, 56)

同様のタプルを``pendulum.datetime``に与え、タイムゾーン情報を持った``tz-aware``なオブジェクトが作成できます。
デフォルトでは``UTC``が設定されます。``tz="Asia/Tokyo``オプションを追加すると、日本時刻として作成できます。

In [8]:
dt = pendulum.datetime(2024, 1, 2, 3, 45, 56)
print(dt)
dt

2024-01-02 03:45:56+00:00


DateTime(2024, 1, 2, 3, 45, 56, tzinfo=Timezone('UTC'))

``pendulum.local``を使うと、実行環境のタイムゾーンを持った日時オブジェクトが作成できます。
これは``pendulum.datetime(tz="local")``のエイリアスです。

In [9]:
dt = pendulum.local(2024, 1, 2, 3, 45, 56)
print(dt)
dt

2024-01-02 03:45:56+09:00


DateTime(2024, 1, 2, 3, 45, 56, tzinfo=Timezone('Asia/Tokyo'))

# 日付を変換したい

ある日時オブジェクトを、別の並びの文字列に変換したいことがあります。
たとえばRSSフィードの日時は``曜日（短縮）, 日 月名（短縮） 年 時:分:秒 +Z``であったり、
ISO8601形式の日時は``年-月-日T時:分:秒+Z``であったりします。

``datetime``モジュールの場合、変換後の文字列は自分で指定する必要があります。
``pendulum``パッケージの場合、よく使われる日時の文字列はプリセットされています。
なるべくプリセットされたものを使うのがよいでしょう。

In [10]:
# RSS形式
dt = datetime.datetime(2024, 1, 2, 3, 45, 56)
dt.strftime("%a, %d %b %Y %H:%M:%S")

'Tue, 02 Jan 2024 03:45:56'

In [11]:
# ISO8601形式
dt = datetime.datetime(2024, 1, 2, 3, 45, 56)
dt.strftime("%Y-%m-%dT%H:%M:%S")

'2024-01-02T03:45:56'

In [12]:
# RSS形式
dt = pendulum.local(2024, 1, 2, 3, 45, 56)
dt.to_rss_string()

'Tue, 02 Jan 2024 03:45:56 +0900'

In [13]:
# ISO8601形式
dt = pendulum.local(2024, 1, 2, 3, 45, 56)
dt.to_iso8601_string()

'2024-01-02T03:45:56+09:00'

In [14]:
# ISO8601形式
dt = pendulum.local(2024, 1, 2, 3, 45, 56)
dt.isoformat()

'2024-01-02T03:45:56+09:00'

# 日付を読み込みたい

すでに保存されたデータから日付を読み込み日時オブジェクトに変換したい場合もよくあります。

1. ``datetime.datetime.strptime``
2. ``pendulum.from_format``
3. ``pendulum.parse``

In [15]:
timestamp = "2024-01-02 03:45:56"
apache_time = "01/Sep/2012:06:05:11 +0000"

``datetime``モジュールでは、``datetime.datetime.strptime``を使っいます。

In [16]:
datetime.datetime.strptime(timestamp, "%Y-%m-%d %H:%M:%S")

datetime.datetime(2024, 1, 2, 3, 45, 56)

In [17]:
datetime.datetime.strptime(apache_time, "%d/%b/%Y:%H:%M:%S +0000")

datetime.datetime(2012, 9, 1, 6, 5, 11)

``pendulum``パッケージでは``pendulum.from_format``を使います。
日時の指定子が``datetime``モジュールと異なる点に注意してください。
利用可能な日付トークンは https://pendulum.eustace.io/docs/#tokens で確認できます。

In [18]:
pendulum.from_format(timestamp, "YYYY-MM-DD HH:mm:ss")

DateTime(2024, 1, 2, 3, 45, 56, tzinfo=Timezone('UTC'))

In [19]:
pendulum.from_format(apache_time, "DD/MMM/YYYY:HH:mm:ss ZZ")

DateTime(2012, 9, 1, 6, 5, 11, tzinfo=FixedTimezone(0, name="+00:00"))

In [20]:
pendulum.from_format(timestamp, "YYYY-MM-DD HH:mm:ss", tz="local")

DateTime(2024, 1, 2, 3, 45, 56, tzinfo=Timezone('Asia/Tokyo'))

In [21]:
pendulum.from_format(apache_time, "DD/MMM/YYYY:HH:mm:ss ZZ", tz="local")

DateTime(2012, 9, 1, 6, 5, 11, tzinfo=FixedTimezone(0, name="+00:00"))

``pendulum.parse``で一般的な日付フォーマットをパースできます。
タイムゾーンが指定されていない場合は、``UTC``になります。

In [22]:
pendulum.parse(timestamp)

DateTime(2024, 1, 2, 3, 45, 56, tzinfo=Timezone('UTC'))

In [23]:
pendulum.parse(timestamp, tz="local")

DateTime(2024, 1, 2, 3, 45, 56, tzinfo=Timezone('Asia/Tokyo'))